In [ ]:
#######NEW PROJECT########
#QUESTION: compare the actual data graph population to the graph without the HIV/AIDS epidemic 
#          so that we can see by what factor the population differs in two situations

import random

In [ ]:
#DA = Deaths due to AIDS

#years: 2000-2017
#btw randomized the DA because there is a range for each DA
#I can add more countries so far I just added two
angolaDA = [random.randit(2600,5900), random.randit(3100 - 6800), random.randit(3700 - 7500), random.randit(4400 - 8300),
            random.randit(5100 - 9500), random.randit(5800 - 10000), random.randit(6300 - 11000), random.randit(6400 - 11000),
            random.randit(6700 - 12000), random.randit(7000 - 13000), random.randit(7300 - 13000), random.randit(7700 - 14000), 
            random.randit(8200 - 15000), random.randit(8500 - 16000), random.randit(7800 - 14000), random.randit(7600 - 14000),
            random.randit(8100 - 15000), random.randit(9500 - 17000)]
burkinafasoDA = [random.randit(11000 - 18000), random.randit(11000 - 18000), random.randit(11000 - 17000), random.randit(11000 - 17000),
                 random.randit(10000 - 15000), random.randit(9000 - 14000), random.randit(7400 - 12000), random.randit(6000 - 11000),
                 random.randit(5200 - 9900), random.randit(4200 - 8800), random.randit(3300 - 7500), random.randit(2700 - 6800), 
                 random.randit(2500 - 5900), random.randit(2600 - 6000), random.randit(2600 - 5800), random.randit(2400 - 5300),
                 random.randit(2100 - 4700), random.randit(1900 - 4100)]

#population from 2000-2017
angolaPOP = [16440924, 16983266, 17572649, 18203369, 18865716, 19552542, 20262399, 20997687, 21759420, 22549547, 23369131, 
             24218565, 25096150,25998340, 27859305, 28813463, 29784193]
burkinafasoPOP = [11607942, 11944587, 122293100, 12654621, 13030569, 13421930, 13829177, 14252021, 14689726, 15141099, 
                  15605217, 16081904, 16571216, 17072723, 17585977, 18110624, 18646433, 19193382]

#BR = birth rate
#it's the birth rate per 1000 people
angolaBR = [49.0, 48.8, 48.5, 48.3, 48.0, 47.7, 47.4, 46.9, 46.5, 45.9, 45.3, 44.7, 44.1, 43.4, 42.9, 42.3, 41.8]
burkinafasoBR = [46.3, 46.0, 45.8, 45.5, 45.2, 44.8, 44.4, 43.9, 43.4, 42.9, 42.3 41.7, 41.1, 40.5, 40.0, 39.4, 38.9]

#DR = death rate
#it's the death rate per 1000 people
angolaDR = [17.9, 17.0, 16.2, 15.4, 14.5, 13.8, 13.0, 12.3, 11.7, 11.0, 10.5, 9.9, 9.5, 9.2, 8.9, 8.7, 8.5]
burkinafasoDR = [15.5, 15.2, 14.8, 14.4, 13.9, 13.3, 12.8, 12.2, 11.7, 11.1, 10.6, 10.2, 9.8, 9.4, 9.1, 8.8, 8.6]